## Manual implementation of Neural Network


'''

Operation - aka activation function
Input Nodes - aka
Output Nodes - aka


Operation Class
    * Input Nodes
    * Output Nodes
    * Global Default Graph Variable
    * Compute
        * Overwritten by extended classes
'''

In [216]:
import numpy as np

'''
Graph is a list of nodes

'''

class Operation:

    def __init__(self, input_nodes=[]):

        self.input_nodes = input_nodes
        self.output_nodes = []

        for node in input_nodes:
            node.output_nodes.append(self)

        _default_graph.operations.append(self)

    def compute(self):
        pass

In [217]:
class Add(Operation):

    def __init__(self, x, y):
        super().__init__([x, y])

    def compute(self,x_var, y_var):
        self.inputs = [x_var, y_var]

        return x_var + y_var


In [218]:
class Multiply(Operation):

    def __init__(self, x, y):

        super().__init__([x, y])

    def compute(self,x_var, y_var):

        self.inputs = [x_var, y_var]
        return x_var * y_var

In [219]:
class MatMul(Operation):

    def __init__(self, x, y):
        super().__init__([x, y])

    def compute(self,x_var, y_var):
        self.inputs = [x_var, y_var]

        return x_var.dot(y_var) # dot product of a matrix

## Info

'''
Variables - Changeable parameter of Graph.

Placeholders - An 'empty' node that needs a value to be provided to compute output.

Graphs - Global variable connection variables and placeholders to operations.
'''

In [220]:
class Placeholder():

    def __init__(self):

        self.output_nodes = []

        _default_graph.placeholders.append(self)

In [221]:
class Variable():

    def __init__(self,initial_value=None):

        self.value = initial_value
        self.output_nodes = []

        _default_graph.variables.append(self)

In [222]:
class Graph():

    def __init__(self):

        self.operations = []
        self.placeholders = []
        self.variables = []

    def set_as_default(self):

        global _default_graph
        _default_graph = self

## A basic graph

'''
z = Ax + b

A = 10 - input 1
b = 1 - input 2

z = 10x + 1 - x is the place holder

z is the output
'''

In [223]:
g = Graph()

In [224]:
g.set_as_default()

In [225]:
A = Variable(10)

In [226]:
b = Variable(1)

In [227]:
x = Placeholder()

In [228]:
y = Multiply(A,x)

In [229]:
z = Add(y,b)

## Session

In [230]:
'''
We will use a PostOrder Tree Traversal to make sure we execute the nodes in the correct order.
'''

def traverse_postorder(operation):
    """
    PostOrder Traversal of Nodes. Basically makes sure computations are done in
    the correct order (Ax first , then Ax + b). Feel free to copy and paste this code.
    It is not super important for understanding the basic fundamentals of deep learning.
    """
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)

    recurse(operation)
    return nodes_postorder

In [231]:
class Session():

    def run(self,operation,feed_dict={}):

        node_postorder = traverse_postorder(operation)

        for node in node_postorder:

            if type(node) == Placeholder:
                node.output = feed_dict[node]

            elif type(node) == Variable:
                node.output = node.value

            else:
                node.inputs = [input_node.output for input_node in node.input_nodes]

                node.output = node.compute(*node.inputs)# we don't know the size of this list so * means args

            if type(node.output) == list:
                node.output = np.array(node.output)

        return operation.output



In [232]:
sess = Session()

In [233]:
result = sess.run(operation=z,feed_dict={x:10})

In [234]:
result

101

## bravo

10x + 1

x = 10

10 * 10 + 1 = 101

## Now Multiply with a matrix

In [235]:
g = Graph()

g.set_as_default()

A = Variable([[30,40],
              [50,10]])
b = Variable([1,2,])

x = Placeholder()

y = MatMul(A,x)

z = Add(y,b)

In [236]:
sess = Session()

In [237]:
sess.run(operation=z,feed_dict={x:10})

array([[301, 402],
       [501, 102]])